<a href="https://colab.research.google.com/github/Hriman-Mahanta/Flood_Image_Segmentation/blob/main/IP_Project_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def get_tuple_files(path):
    # Get all image files in the given path
    files = get_image_files(path)
    
    # Filter the files to keep only those ending with 'vhr.tif'
    files = [f for f in files if f.name.endswith('vhr.tif')]
    
    # Return the filtered list of files
    return files

# Call the function to get the tuple files
files = get_tuple_files(path)

In [ ]:
def label_func_flooded(x):
    return x.parents[0] / 'flooded1m.tif'

In [ ]:
# Create the directory DATA_HAR to store the preprocessed images
os.makedirs("DATA_HAR", exist_ok=True)

In [ ]:
path = Path('/content/drive/MyDrive/dataset')
files = get_tuple_files(path)

In [ ]:
# Process each image in the dataset
for j, fil in enumerate(files, start=1):
    im = PILImage.create(fil)
    mask = PILMask.create(label_func_flooded(fil))
    im = image2tensor(im.reshape(1024, 1024, resample=Image.BICUBIC)).cuda()
    mask = image2tensor(mask.reshape(1024, 1024, resample=Image.BICUBIC)).cuda()
    
    size = 512  # patch size
    stride = 512  # patch stride
    
    # Extract patches from the image and mask
    img = im.unfold(1, size, stride).unfold(2, size, stride)
    img = img.reshape((3, 4, 512, 512))
    mask = mask.unfold(1, size, stride).unfold(2, size, stride)
    mask = mask.reshape((4, 512, 512))
    img = img.permute(1, 2, 3, 0)
    
    # Save each patch as a separate image file
    for i in range(4):
        img_patch = PILImage.create(img[i].cpu())
        mask_patch = PILMask.create(mask[i].cpu())
        img_patch.save(f"DATA_HAR/{fil.stem}_{i}_vhr.tif")
        mask_patch.save(f"DATA_HAR/{fil.stem}_{i}_mask_flood.tif")
        
    print(j)


In [ ]:
# Display example images
im = PILImage.create(files[0]).reshape(512, 512, resample=3)
mask_flood = PILMask.create(label_func_flooded(files[0])).reshape(512, 512, resample=3)

fig, axarr = plt.subplots(1, 2, figsize=(25, 25), sharex=True, sharey=True)
axarr[0].imshow(im)
axarr[0].set_title('VHR Image')
axarr[1].imshow(mask_flood)
axarr[1].set_title('Flood Damage Mask Patch')

In [ ]:
# Copy the directory DATA_HAR to destination folder
destination_folder = '/content/drive/MyDrive/Refined_Images'
os.makedirs(destination_folder, exist_ok=True)
os.system(f'cp -r DATA_HAR/* {destination_folder}')

In [ ]:
# Remove the temporary DATA_HAR directory
os.system('rm -r DATA_HAR')